In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.polars
import altair as alt

In [2]:
df_path = r"F:\Datasets\CSV datasets\laptop_prices.csv"

In [3]:
df = pl.read_csv(df_path, has_header=True, try_parse_dates=True)

In [4]:
df

Brand,Processor,RAM (GB),Storage,GPU,Screen Size (inch),Resolution,Battery Life (hours),Weight (kg),Operating System,Price ($)
str,str,i64,str,str,f64,str,f64,f64,str,f64
"""Apple""","""AMD Ryzen 3""",64,"""512GB SSD""","""Nvidia GTX 1650""",17.3,"""2560x1440""",8.9,1.42,"""FreeDOS""",3997.07
"""Razer""","""AMD Ryzen 7""",4,"""1TB SSD""","""Nvidia RTX 3080""",14.0,"""1366x768""",9.4,2.57,"""Linux""",1355.78
"""Asus""","""Intel i5""",32,"""2TB SSD""","""Nvidia RTX 3060""",13.3,"""3840x2160""",8.5,1.74,"""FreeDOS""",2673.07
"""Lenovo""","""Intel i5""",4,"""256GB SSD""","""Nvidia RTX 3080""",13.3,"""1366x768""",10.5,3.1,"""Windows""",751.17
"""Razer""","""Intel i3""",4,"""256GB SSD""","""AMD Radeon RX 6600""",16.0,"""3840x2160""",5.7,3.38,"""Linux""",2059.83
…,…,…,…,…,…,…,…,…,…,…
"""Acer""","""Intel i3""",4,"""2TB SSD""","""Nvidia RTX 2060""",17.3,"""1366x768""",11.5,1.58,"""macOS""",704.82
"""Asus""","""Intel i3""",4,"""2TB SSD""","""AMD Radeon RX 6800""",16.0,"""1366x768""",9.5,2.14,"""Linux""",775.59
"""Razer""","""AMD Ryzen 9""",4,"""2TB SSD""","""AMD Radeon RX 6600""",15.6,"""2560x1440""",8.2,2.05,"""Linux""",2789.46


In [5]:
df = df.with_columns(
    pl.col('Processor').str.contains('AMD').alias('AMD CPU'),
    pl.col('Processor').str.contains('Intel').alias('Intel CPU')
)

In [6]:
amd_cpu_price = df.filter(
    pl.col('AMD CPU') == True
).select(
    pl.col('Price ($)')
).mean()
amd_cpu_price

Price ($)
f64
2184.162107


In [7]:
intel_cpu_price = df.filter(
    pl.col('AMD CPU') == False
).select(
    pl.col('Price ($)')
).mean()

In [8]:
cpu_prices = {
    'CPU Type': ['AMD', 'Intel'],
    'Average Price': [amd_cpu_price['Price ($)'][0], intel_cpu_price['Price ($)'][0]]
}

plot_df = pl.DataFrame(cpu_prices)

plot = plot_df.hvplot.bar(
    x='CPU Type',
    y='Average Price',
    title='Average Laptop Prices: AMD vs Intel',
    ylabel='Price ($)',
    color=['#ED1C24', '#0071C5']
)

plot

:Bars   [CPU Type]   (Average Price)

In [18]:
df.with_columns(pl.col('Price ($)').map_elements(lambda x: x * 2, return_dtype=pl.Float64).alias('Double Price'))

Brand,Processor,RAM (GB),Storage,GPU,Screen Size (inch),Resolution,Battery Life (hours),Weight (kg),Operating System,Price ($),AMD CPU,Intel CPU,Double Price
str,str,i64,str,str,f64,str,f64,f64,str,f64,bool,bool,f64
"""Apple""","""AMD Ryzen 3""",64,"""512GB SSD""","""Nvidia GTX 1650""",17.3,"""2560x1440""",8.9,1.42,"""FreeDOS""",3997.07,true,false,7994.14
"""Razer""","""AMD Ryzen 7""",4,"""1TB SSD""","""Nvidia RTX 3080""",14.0,"""1366x768""",9.4,2.57,"""Linux""",1355.78,true,false,2711.56
"""Asus""","""Intel i5""",32,"""2TB SSD""","""Nvidia RTX 3060""",13.3,"""3840x2160""",8.5,1.74,"""FreeDOS""",2673.07,false,true,5346.14
"""Lenovo""","""Intel i5""",4,"""256GB SSD""","""Nvidia RTX 3080""",13.3,"""1366x768""",10.5,3.1,"""Windows""",751.17,false,true,1502.34
"""Razer""","""Intel i3""",4,"""256GB SSD""","""AMD Radeon RX 6600""",16.0,"""3840x2160""",5.7,3.38,"""Linux""",2059.83,false,true,4119.66
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Acer""","""Intel i3""",4,"""2TB SSD""","""Nvidia RTX 2060""",17.3,"""1366x768""",11.5,1.58,"""macOS""",704.82,false,true,1409.64
"""Asus""","""Intel i3""",4,"""2TB SSD""","""AMD Radeon RX 6800""",16.0,"""1366x768""",9.5,2.14,"""Linux""",775.59,false,true,1551.18
"""Razer""","""AMD Ryzen 9""",4,"""2TB SSD""","""AMD Radeon RX 6600""",15.6,"""2560x1440""",8.2,2.05,"""Linux""",2789.46,true,false,5578.92


In [40]:
df.pivot(
    index='Brand',
    on='Storage',
    values='Price ($)',
    aggregate_function='mean'
)

Brand,512GB SSD,1TB SSD,2TB SSD,256GB SSD,1TB HDD
str,f64,f64,f64,f64,f64
"""Apple""",3095.289403,2937.319955,2920.519228,3025.939919,2609.298914
"""Razer""",2648.8576,2851.597091,2792.696413,2808.24448,2337.868551
"""Asus""",1994.06396,1959.418465,1955.284249,1910.389766,1815.225417
"""Lenovo""",2016.879461,2039.072304,2018.83893,2191.726284,1630.188053
"""Acer""",1945.202753,1955.700766,1964.069614,1978.603054,1717.818839
"""Dell""",2011.450667,2058.401532,1982.006387,2113.511443,1688.006776
"""Microsoft""",2009.473061,1956.780043,2045.612591,2234.29214,1607.897864
"""HP""",2042.168139,2060.061941,1978.85129,1911.91898,1702.365612
"""Samsung""",1945.826535,2152.040345,2019.810448,1968.066604,1690.45693


In [44]:
df.select(pl.col('Processor').str.len_chars())

Processor
u32
11
11
8
8
8
…
8
8
11


In [51]:
df.with_columns(pl.col('Price ($)').cum_max().over('Processor'))

Brand,Processor,RAM (GB),Storage,GPU,Screen Size (inch),Resolution,Battery Life (hours),Weight (kg),Operating System,Price ($),AMD CPU,Intel CPU
str,str,i64,str,str,f64,str,f64,f64,str,f64,bool,bool
"""Apple""","""AMD Ryzen 3""",64,"""512GB SSD""","""Nvidia GTX 1650""",17.3,"""2560x1440""",8.9,1.42,"""FreeDOS""",3997.07,true,false
"""Razer""","""AMD Ryzen 7""",4,"""1TB SSD""","""Nvidia RTX 3080""",14.0,"""1366x768""",9.4,2.57,"""Linux""",1355.78,true,false
"""Asus""","""Intel i5""",32,"""2TB SSD""","""Nvidia RTX 3060""",13.3,"""3840x2160""",8.5,1.74,"""FreeDOS""",2673.07,false,true
"""Lenovo""","""Intel i5""",4,"""256GB SSD""","""Nvidia RTX 3080""",13.3,"""1366x768""",10.5,3.1,"""Windows""",2673.07,false,true
"""Razer""","""Intel i3""",4,"""256GB SSD""","""AMD Radeon RX 6600""",16.0,"""3840x2160""",5.7,3.38,"""Linux""",2059.83,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Acer""","""Intel i3""",4,"""2TB SSD""","""Nvidia RTX 2060""",17.3,"""1366x768""",11.5,1.58,"""macOS""",5543.63,false,true
"""Asus""","""Intel i3""",4,"""2TB SSD""","""AMD Radeon RX 6800""",16.0,"""1366x768""",9.5,2.14,"""Linux""",5543.63,false,true
"""Razer""","""AMD Ryzen 9""",4,"""2TB SSD""","""AMD Radeon RX 6600""",15.6,"""2560x1440""",8.2,2.05,"""Linux""",10807.88,true,false


In [61]:
df.with_columns(pl.col('Price ($)').rolling_mean(window_size=3).over('GPU'))

Brand,Processor,RAM (GB),Storage,GPU,Screen Size (inch),Resolution,Battery Life (hours),Weight (kg),Operating System,Price ($),AMD CPU,Intel CPU
str,str,i64,str,str,f64,str,f64,f64,str,f64,bool,bool
"""Apple""","""AMD Ryzen 3""",64,"""512GB SSD""","""Nvidia GTX 1650""",17.3,"""2560x1440""",8.9,1.42,"""FreeDOS""",null,true,false
"""Razer""","""AMD Ryzen 7""",4,"""1TB SSD""","""Nvidia RTX 3080""",14.0,"""1366x768""",9.4,2.57,"""Linux""",null,true,false
"""Asus""","""Intel i5""",32,"""2TB SSD""","""Nvidia RTX 3060""",13.3,"""3840x2160""",8.5,1.74,"""FreeDOS""",null,false,true
"""Lenovo""","""Intel i5""",4,"""256GB SSD""","""Nvidia RTX 3080""",13.3,"""1366x768""",10.5,3.1,"""Windows""",null,false,true
"""Razer""","""Intel i3""",4,"""256GB SSD""","""AMD Radeon RX 6600""",16.0,"""3840x2160""",5.7,3.38,"""Linux""",null,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Acer""","""Intel i3""",4,"""2TB SSD""","""Nvidia RTX 2060""",17.3,"""1366x768""",11.5,1.58,"""macOS""",1173.643333,false,true
"""Asus""","""Intel i3""",4,"""2TB SSD""","""AMD Radeon RX 6800""",16.0,"""1366x768""",9.5,2.14,"""Linux""",1181.693333,false,true
"""Razer""","""AMD Ryzen 9""",4,"""2TB SSD""","""AMD Radeon RX 6600""",15.6,"""2560x1440""",8.2,2.05,"""Linux""",2725.71,true,false


In [62]:
df.vstack(df)

Brand,Processor,RAM (GB),Storage,GPU,Screen Size (inch),Resolution,Battery Life (hours),Weight (kg),Operating System,Price ($),AMD CPU,Intel CPU
str,str,i64,str,str,f64,str,f64,f64,str,f64,bool,bool
"""Apple""","""AMD Ryzen 3""",64,"""512GB SSD""","""Nvidia GTX 1650""",17.3,"""2560x1440""",8.9,1.42,"""FreeDOS""",3997.07,true,false
"""Razer""","""AMD Ryzen 7""",4,"""1TB SSD""","""Nvidia RTX 3080""",14.0,"""1366x768""",9.4,2.57,"""Linux""",1355.78,true,false
"""Asus""","""Intel i5""",32,"""2TB SSD""","""Nvidia RTX 3060""",13.3,"""3840x2160""",8.5,1.74,"""FreeDOS""",2673.07,false,true
"""Lenovo""","""Intel i5""",4,"""256GB SSD""","""Nvidia RTX 3080""",13.3,"""1366x768""",10.5,3.1,"""Windows""",751.17,false,true
"""Razer""","""Intel i3""",4,"""256GB SSD""","""AMD Radeon RX 6600""",16.0,"""3840x2160""",5.7,3.38,"""Linux""",2059.83,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Acer""","""Intel i3""",4,"""2TB SSD""","""Nvidia RTX 2060""",17.3,"""1366x768""",11.5,1.58,"""macOS""",704.82,false,true
"""Asus""","""Intel i3""",4,"""2TB SSD""","""AMD Radeon RX 6800""",16.0,"""1366x768""",9.5,2.14,"""Linux""",775.59,false,true
"""Razer""","""AMD Ryzen 9""",4,"""2TB SSD""","""AMD Radeon RX 6600""",15.6,"""2560x1440""",8.2,2.05,"""Linux""",2789.46,true,false


In [72]:
df_sorted = df.sort('Storage')
df_sorted.group_by('Storage').agg(pl.col('Brand').count().alias('Brand_Count'))


Storage,Brand_Count
str,u32
"""1TB HDD""",2302
"""256GB SSD""",2318
"""1TB SSD""",2313
"""2TB SSD""",2415
"""512GB SSD""",2420
